In [1]:
include("AZP_env.jl")

score_test (generic function with 1 method)

In [2]:
function calc_Kt_sym(history::Vector{Int}, env::Env)
    MV = []
    his = copy(history)
    #println(length(his))
    for it in 1:length(his)
        sw = pop!(his)
        if(sw==1)
            push!(MV, env.H_0)
        elseif(sw==2)
            push!(MV, env.V_t*sx)
        elseif(sw==3)
            A = pop!(MV)
            B = pop!(MV)
            C = A + B
            push!(MV, C)
        elseif(sw==4)
            A = pop!(MV)
            B = pop!(MV)
            C = -1im*(A*B - B*A)
            push!(MV, C)
        elseif(sw==5)
            A = pop!(MV)
            B = pop!(MV)
            C = (A*B + B*A)/2
            push!(MV, C)
        elseif(sw==6)
            A = pop!(MV)
            try
                S = A.subs(x, t[1])-A.subs(x, t[env.t_step//4])
                if(S==zeros(env.HS_size, env.HS_size))
                    B = A
                else
                    B = A.integrate(x)/env.Ω
                end
            catch
                B = A
            end
            push!(MV, B)
        end
        #@show MV
    end
    return MV[end]
end

calc_Kt_sym (generic function with 1 method)

In [3]:
en = init_Env_quick(["15", "64", "5", "0.25"], Any[0.3, Int(72), 1.0])

Env(15, 200, 2, 3, 1, 6, 90, 64, 7, 5, 1000, 200, 500, 1, 1.0f-5, 0.9f0, 512, 0.3f0, 0.25f0, 0.1f0, 1.0f0, 100, 4, 10.0f0, 0.4f0, 1.0f0, 0.7f0, 0.5f0, ComplexF32[-2.0f0 + 0.0f0im 0.0f0 + 0.0f0im 0.0f0 + 0.0f0im -0.7f0 + 0.0f0im; 0.0f0 - 0.0f0im 1.0f0 + 0.0f0im -0.7f0 + 0.0f0im 0.0f0 + 0.0f0im; 0.0f0 - 0.0f0im -0.7f0 - 0.0f0im 1.0f0 + 0.0f0im 0.0f0 + 0.0f0im; -0.7f0 - 0.0f0im 0.0f0 - 0.0f0im 0.0f0 - 0.0f0im 0.0f0 + 0.0f0im], ComplexF32[0.0f0 + 0.0f0im -0.4f0 + 0.0f0im -0.4f0 + 0.0f0im 0.0f0 + 0.0f0im; -0.4f0 - 0.0f0im 0.0f0 + 0.0f0im 0.0f0 + 0.0f0im -0.4f0 + 0.0f0im; -0.4f0 - 0.0f0im 0.0f0 - 0.0f0im 0.0f0 + 0.0f0im -0.4f0 + 0.0f0im; 0.0f0 - 0.0f0im -0.4f0 - 0.0f0im -0.4f0 - 0.0f0im 0.0f0 + 0.0f0im], 0.0062831854f0, 72, 1.0f0, 1.0f-6)

In [4]:
t = collect(0:en.Ω*en.dt:2pi)

100-element Vector{Float64}:
 0.0
 0.06283185631036758
 0.12566371262073517
 0.18849556893110275
 0.25132742524147034
 0.3141592815518379
 0.3769911378622055
 0.4398229941725731
 0.5026548504829407
 0.5654867067933083
 ⋮
 5.71769892424345
 5.780530780553818
 5.843362636864185
 5.906194493174553
 5.9690263494849205
 6.031858205795288
 6.094690062105656
 6.157521918416023
 6.220353774726391

In [27]:
#hist = [6, 6, 6, 6, 3, 1, 4, 2, 1]
#hist = [6, 6, 6, 3, 2, 6, 5, 1, 5, 2, 6, 4, 2, 1]
hist = [5, 6, 3, 2, 4, 2, 5, 1, 5, 2, 2, 2]
#hist = [1]

@time Ks = calc_Kt_sym(hist, en)
@time println(calc_score(hist, en))

In [17]:
K0 = convert(Matrix{ComplexF32}, Ks.subs(x,t[1]))

4×4 Matrix{ComplexF32}:
 0.0+0.0im  0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im  0.0+0.0im  0.0+0.0im

In [18]:
@time Kt = [Hermitian(convert(Matrix{ComplexF32}, Ks.subs(x,t[i]))-K0) for i in 1:en.t_step]
println("")

  0.327439 seconds (228.18 k allocations: 13.521 MiB, 37.75% compilation time)

